#### Using the OSM-APIs to create a list so we can visualize all the cafes in OpenStreetMap

In [19]:
import requests
print(requests.__version__)
from random import randint
import time

2.32.3


In [23]:
def getOSM_data_from_Cafes(cafe, geo_list, proxies, sleeptime, use_proxy=False):
    nominatim_url = "https://nominatim.openstreetmap.org/search"
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; rv:85.0) Gecko/20100101 Firefox/85.0"
    ]
    
    headers = {"User-Agent": user_agents[randint(0, len(user_agents) - 1)]}
    proxy = {"http": None, "https": None}
    
    if use_proxy and proxies:
        proxy_to_use = proxies[randint(0, len(proxies) - 1)]
        proxy = {"http": proxy_to_use, "https": proxy_to_use}
    
    if not cafe:
        return geo_list

    time.sleep(sleeptime)
    
    try:
        def fetch_osm_data(query):
            params = {"q": query, "format": "json", "addressdetails": 1, "limit": 1}
            response = requests.get(nominatim_url, params=params, headers=headers, proxies=proxy if use_proxy else None, timeout=10)
            response.raise_for_status()
            return response.json()

        # First, try with the cafe name and address
        data = fetch_osm_data(f"{cafe[0]}, {cafe[1]}")

        if not data:  # If no data, fallback to address only
            data = fetch_osm_data(cafe[1])

        if data:
            osm_url = f"https://www.openstreetmap.org/{data[0]['osm_type']}/{data[0]['osm_id']}"
            geo_list.append({
                "name": cafe[0],
                "osm_url": osm_url,
                "coordinates": [float(data[0]["lon"]), float(data[0]["lat"])]
            })
            print(f"Data saved for {cafe[0]} & {cafe[1]} with proxy {proxy}")
        else:
            geo_list.append({"name": cafe[0], "osm_url": "Not Found", "coordinates": [0.0, 0.0]})
            print(f"No data found for {cafe[0]} & {cafe[1]} with proxy {proxy}")

        return geo_list

    except requests.RequestException as e:
        print(f"Request failed for {cafe[0]} & {cafe[1]} with proxy {proxy}. Error: {e}")
        if use_proxy and proxies:
            proxies.remove(proxy["http"])  # Remove the faulty proxy
        return getOSM_data_from_Cafes(cafe, geo_list, proxies, use_proxy)

In [ ]:
# Example call
geo_list = []
proxies = ["http://54.152.3.36:80", "http://63.35.64.177:3128"]
cafes = ["Můj šálek kávy", "Křižíkova 105, 186 00 Prague, Czech Republic"]
getOSM_data_from_Cafes(cafes, geo_list, proxies, 2)

In [24]:
proxies = [
    "http://195.35.2.231:80",
"http://72.10.160.90:20975",
"http://103.152.247.80:8088",
]

In [ ]:
geo = []
cafe = ["Coffee square Lednice","Valtická 253, 691 44 Lednice, Czech Republic"]
cafe_2 = ["Můj šálek kávy", "Křižíkova 105, 186 00 Prague, Czech Republic"]
cafe_3 = ["Elias Coffee Shop","Štěpánská 623/40, 110 00 Prague, Czech Republic"]
cafe4 = [
                "Rush Rush",
                "Lange Altaarstraat 29, 2018 Antwerp, Belgium"
            ]
cafe5 = [
                "My Little Melbourne",
                "Madách Imre út 3, 1075 Budapest, Hungary"
            ]
cafe6 = [
                "Sugar Blue",
                "4 Rue de l'Arche Sèche, 44000 Nantes, France"
            ]
cafe7 = [
                "Goldhahn & Sampson",
                "Wilmersdorfer Straße 102/103, 10629 Berlin, Germany"
            ]
        
getOSM_data_from_Cafes(cafe, geo, proxies, use_proxy=False)
getOSM_data_from_Cafes(cafe_2, geo, proxies, use_proxy=False)
getOSM_data_from_Cafes(cafe_3, geo, proxies, use_proxy=False)
getOSM_data_from_Cafes(cafe4, geo, proxies, use_proxy=False)
getOSM_data_from_Cafes(cafe5, geo, proxies, use_proxy=False)
getOSM_data_from_Cafes(cafe6, geo, proxies, use_proxy=False)
getOSM_data_from_Cafes(cafe7, geo, proxies, use_proxy=False)

##### Iterate through the dictionary with all cafes and addresses and find the corresponding OpenStreetMaps-ID and store it in a list

In [27]:
cafes_OSM_id = []
euCoffeeTripData = {
    "czech republic": {
        "https://europeancoffeetrip.com/prague/": {
            "https://europeancoffeetrip.com/cafe/mujsalekkavy/": [
                "Můj šálek kávy",
                "Křižíkova 105, 186 00 Prague, Czech Republic"
            ],
            "https://europeancoffeetrip.com/cafe/elias-coffeeshop-prague/": [
                "Elias Coffee Shop",
                "Štěpánská 623/40, 110 00 Prague, Czech Republic"
            ],
        }
    },
    "spain": {
        "https://europeancoffeetrip.com/barcelona/": {
            "https://europeancoffeetrip.com/cafe/aeroplano-barcelona/": [
                "Aeroplano.Coffee",
                "C/ de Muntaner, 423, 08021 Barcelona, Spain"
            ],
            "https://europeancoffeetrip.com/cafe/dorigencoffee-barcelona/": [
                "D·Origen Coffee Roasters",
                "C/ de Casp, 48, 08010 Barcelona, Spain"
            ]
            }
    }
    
}

In [42]:
#Import of the saved euCoffeeTripData - Dictionary -> just so safe requests against the Website and not create it everytime new
import ast

with open('Dict-saved5.txt', 'r', encoding='utf-8') as file:
    dict_data = file.read()

# Convert the string representation into a Python dictionary
euCoffeeTripData = ast.literal_eval(dict_data)

print(euCoffeeTripData)
#should be ~210 cities distributed over all countries

{'czech republic': {'https://europeancoffeetrip.com/prague/': {'https://europeancoffeetrip.com/cafe/mujsalekkavy/': ['Můj šálek kávy', 'Křižíkova 105, 186 00 Prague, Czech Republic'], 'https://europeancoffeetrip.com/cafe/elias-coffeeshop-prague/': ['Elias Coffee Shop', 'Štěpánská 623/40, 110 00 Prague, Czech Republic'], 'https://europeancoffeetrip.com/cafe/tomcatcoffee-prague/': ['Tomcat coffee', 'Benešova 1273/1, 152 00 Prague, Czech Republic'], 'https://europeancoffeetrip.com/cafe/spellcoffee-prague/': ['Spell Coffee', 'Jindřišská 939/20, 110 00 Prague, Czech Republic'], 'https://europeancoffeetrip.com/cafe/onesipcoffee-prague/': ['onesip coffee', 'Haštalská 755 /15, 110 00 Prague, Czech Republic'], 'https://europeancoffeetrip.com/cafe/dobroteka-prague/': ['Dobrotéka Žižkov', 'Hartigova 176, 130 00 Prague, Czech Republic'], 'https://europeancoffeetrip.com/cafe/mistralcafe-prague/': ['Mistral café', 'Valentinská 11, 110 00 Prague, Czech Republic'], 'https://europeancoffeetrip.com/cafe

In [43]:
#Import the list with the GeoIDS
import ast

# Open the file and read its contents
with open('List_with_coordinates_1.txt', 'r', encoding='utf-8') as file:
    list_data = file.read()

# Convert the string representation into a Python dictionary or list (depending on the file contents)
cafes_OSM_id = ast.literal_eval(list_data)

# Print the result
print(cafes_OSM_id)

[{'name': 'Můj šálek kávy', 'osm_url': 'https://www.openstreetmap.org/node/1317088234', 'coordinates': [14.455158, 50.0935184]}, {'name': 'Elias Coffee Shop', 'osm_url': 'https://www.openstreetmap.org/node/296492209', 'coordinates': [14.4261752, 50.0799321]}, {'name': 'Tomcat coffee', 'osm_url': 'https://www.openstreetmap.org/node/9104442885', 'coordinates': [14.3654826, 50.0281864]}, {'name': 'Spell Coffee', 'osm_url': 'https://www.openstreetmap.org/node/5654640033', 'coordinates': [14.4288988, 50.0841504]}, {'name': 'onesip coffee', 'osm_url': 'https://www.openstreetmap.org/node/4214305897', 'coordinates': [14.4254402, 50.0913069]}, {'name': 'Dobrotéka Žižkov', 'osm_url': 'https://www.openstreetmap.org/node/296509859', 'coordinates': [14.4800668, 50.0919218]}, {'name': 'Mistral café', 'osm_url': 'https://www.openstreetmap.org/node/3146941029', 'coordinates': [14.4162862, 50.0883139]}, {'name': 'Industra Coffee', 'osm_url': 'https://www.openstreetmap.org/node/296562449', 'coordinates'

In [ ]:
#IP used for this:79.210.191.135
# time.sleep(3)

for country in euCoffeeTripData:
    city_urls = list(euCoffeeTripData[country].keys())
    for city_url in city_urls:
        cafe_urls = list(euCoffeeTripData[country][city_url].keys())
        for cafe_url in cafe_urls:
            
            cafe_data = euCoffeeTripData[country][city_url][cafe_url]
            try:
                cafe_name = cafe_data[0]  
                cafe_address = cafe_data[1]
            except:
                continue
            existing_cafe = next((cafe for cafe in cafes_OSM_id if cafe['name'] == cafe_name), None)
            if existing_cafe and existing_cafe['osm_url'] != "Not Found":
                # Skip this entry as the cafe already has an osm_url
                print(f"Skipping {cafe_name} as it already has an osm_url.")
                continue  # Skip to the next cafe

            
            #print(f"{euCoffeeTripData[country][city_url][cafe_url]}")
            getOSM_data_from_Cafes(euCoffeeTripData[country][city_url][cafe_url], cafes_OSM_id, proxies, 5)
        print(f"#####Actual cafes_OSM_ids: {cafes_OSM_id}")

In [ ]:
#Need To Do!!!
#Second run and filter for all the cafes were no coordinates were stored and try them again!

cafes_OSM_id

In [47]:
import json

with open('List_with_coordinates_2.txt', 'w', encoding='utf-8') as file:
    json.dump(cafes_OSM_id, file, ensure_ascii=False, indent=4)

#### Creation of a OpenStreetMaps list for https://umap.openstreetmap.de/de/

In [48]:
import json
#TODO -> Need to delete the Not found cafes with the coordinates [0.0, 0.0]
# GeoJSON-Format für uMap
geojson = {
    "type": "FeatureCollection",
    "features": []
}

for cafe in cafes_OSM_id:
    if cafe["osm_url"] != "Not Found":
        geojson["features"].append({
            "type": "Feature",
            "properties": {
                "name": cafe["name"],
                "url": cafe["osm_url"]
            },
            "geometry": {
                "type": "Point",
                "coordinates":[cafe['coordinates'][0], cafe['coordinates'][1]]  # Beispielkoordinaten
            }
        })

# GeoJSON-Datei speichern
with open("cafes.geojson", "w") as file:
    json.dump(geojson, file)

print("GeoJSON-Datei erstellt: cafes.geojson")
print(geojson["features"][0]["properties"])


GeoJSON-Datei erstellt: cafes.geojson
{'name': 'Můj šálek kávy', 'url': 'https://www.openstreetmap.org/node/1317088234'}
